In [34]:
import mne
import numpy as np
import pandas as pd
from datetime import datetime

In [35]:
#setting files paths
recording_info_path = "data/recording_metadata.csv"  #big CSV file
seizures_path = "data/1_surf30_seizures.csv"
seizures_df = pd.read_csv('data/1_surf30_seizures.csv')

In [36]:
def load_eeg_file(eeg_file_path):
    """
    Step 1: Read the EEG file using nicolet function
    """
    try:
        raw = mne.io.read_raw_nicolet(eeg_file_path, ch_type='eeg', preload=True)
        print(f"\nEEG file loaded successfully!")
        print(f"Recording length: {raw.times.max():.2f} seconds")
        print(f"Number of channels: {len(raw.ch_names)}")
        return raw
    except Exception as e:
        print(f"Error loading EEG file: {str(e)}")
        return None

In [37]:
def load_seizure_times(seizure_csv_path):
    """
    Step 3: Load seizure times from the CSV file
    """
    try:
        seizures_df = pd.read_csv(seizure_csv_path)
        # Convert onset and offset to datetime
        seizures_df['onset'] = pd.to_datetime(seizures_df['onset'])
        seizures_df['offset'] = pd.to_datetime(seizures_df['offset'])
        print(f"\nSeizure times loaded successfully!")
        print(f"Number of seizures: {len(seizures_df)}")
        return seizures_df
    except Exception as e:
        print(f"Error loading seizure times: {str(e)}")
        return None

In [38]:
# plotting
def find_and_plot_seizure_window(raw, seizure_start_time, recording_start_time):
    """
    Steps 4-5: Find seizure in EEG and plot with 5 minutes before and after
    
    Parameters:
    raw: MNE raw object
    seizure_start_time: datetime object of seizure onset
    recording_start_time: datetime object of recording start
    """
    try:
        # Calculate seizure time relative to recording start
        time_diff = (seizure_start_time - recording_start_time).total_seconds()
        
        # Define window (5 minutes = 300 seconds before and after)
        window_start = max(0, time_diff - 300)  # Don't go before recording start
        window_end = min(raw.times.max(), time_diff + 300)  # Don't go past recording end
        
        # Extract window
        segment = raw.copy().crop(tmin=window_start, tmax=window_end)
        
        # Create plot
        title = f'Seizure at {seizure_start_time.strftime("%Y-%m-%d %H:%M:%S")}'
        fig = segment.plot(
            scalings='auto',
            title=title,
            show_scrollbars=True,
            start=290,  # Start 10 seconds before seizure
            duration=20  # Show 20 seconds by default
        )
        
        return segment
        
    except Exception as e:
        print(f"Error processing seizure window: {str(e)}")
        return None

In [39]:

# Main analysis function
def analyze_seizure_recording(eeg_file_path, seizure_csv_path, metadata_csv_path=None):
    """
    Main function to run all analysis steps
    """
    # Step 1: Load EEG file
    raw = load_eeg_file(eeg_file_path)
    if raw is None:
        return
    
    # Step 2 & 3: Load seizure times
    seizures_df = load_seizure_times(seizure_csv_path)
    if seizures_df is None:
        return
    
    # Load metadata if provided
    if metadata_csv_path:
        metadata_df = pd.read_csv(metadata_csv_path)
        recording_start = pd.to_datetime(metadata_df['start_ts'].iloc[0])
    else:
        # If no metadata, assume recording starts at first seizure minus 10 minutes
        recording_start = seizures_df['onset'].min() - pd.Timedelta(minutes=10)
    
    # Steps 4-5: Process each seizure
    for idx, seizure in seizures_df.iterrows():
        print(f"\nProcessing seizure {idx+1} of {len(seizures_df)}")
        print(f"Seizure onset: {seizure['onset']}")
        print(f"Seizure type: {seizure['classif.']}")
        
        # Find and plot seizure window
        segment = find_and_plot_seizure_window(raw, seizure['onset'], recording_start)
        
        # Optional: wait for user input before processing next seizure
        input("\nPress Enter to continue to next seizure...")

# Example usage
if __name__ == "__main__":
    # File paths
    eeg_file = "data/100102_0075.data"  # Your EEG file
    seizure_csv = "data/1_surf30_seizures.csv"  # Your seizure times CSV
    metadata_csv = "data/pat_1_surf30_file_list.csv"  # Your metadata CSV (optional)
    
    # Run analysis
    analyze_seizure_recording(eeg_file, seizure_csv, metadata_csv)

Reading header...
Reading 0 ... 921599  =      0.000 ...  3599.996 secs...

EEG file loaded successfully!
Recording length: 3600.00 seconds
Number of channels: 29

Seizure times loaded successfully!
Number of seizures: 11

Processing seizure 1 of 11
Seizure onset: 2002-10-01 12:42:40
Seizure type: UC
Error processing seizure window: tmin (92334.0) must be less than tmax (3599.99609375)

Processing seizure 2 of 11
Seizure onset: 2002-10-01 22:47:02
Seizure type: CP
Error processing seizure window: tmin (128596.0) must be less than tmax (3599.99609375)

Processing seizure 3 of 11
Seizure onset: 2002-10-02 07:59:39
Seizure type: UC
Error processing seizure window: tmin (161753.0) must be less than tmax (3599.99609375)

Processing seizure 4 of 11
Seizure onset: 2002-10-02 16:20:06
Seizure type: CP
Error processing seizure window: tmin (191780.0) must be less than tmax (3599.99609375)

Processing seizure 5 of 11
Seizure onset: 2002-10-03 09:46:30
Seizure type: SG
Error processing seizure wi

#insert path after downloading file
mne.io.read_raw_nicolet(path_to_file)